# Importing Dependencies 

In [1]:
import mediapipe as mp
import cv2
import matplotlib.pyplot as plt
import numpy as np

# Defining Functions 

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# Detection model function 

In [3]:
def mediapipe_detection(image,model):
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#     image.flags.writeable = False
    results = model.process(image)
#     image.flags.writeable = True
    return image, results

# Draw landmarks for the model 

In [4]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )

# Sample video testing 

In [5]:
cap = cv2.VideoCapture(0)
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    
    while True:
        
        ret,frame = cap.read()
        
        image, results = mediapipe_detection(frame,pose)
        
        draw_landmarks(image, results)
        
        
        cv2.imshow("Sample video", image)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

# Landmarks for the program

In [6]:
landmarks = results.pose_landmarks.landmark
print(landmarks)

[x: 0.6013217
y: 0.44026327
z: -1.025774
visibility: 0.9998143
, x: 0.6231283
y: 0.38302532
z: -0.9633762
visibility: 0.9996922
, x: 0.63744396
y: 0.38220128
z: -0.96365625
visibility: 0.9996667
, x: 0.6479234
y: 0.382522
z: -0.9639534
visibility: 0.99962187
, x: 0.5760908
y: 0.38875973
z: -0.96709144
visibility: 0.99973965
, x: 0.55946827
y: 0.39139727
z: -0.96682054
visibility: 0.999764
, x: 0.5448307
y: 0.39401445
z: -0.96722746
visibility: 0.99976987
, x: 0.6654395
y: 0.4081721
z: -0.5756644
visibility: 0.99963945
, x: 0.5283865
y: 0.42209625
z: -0.5817853
visibility: 0.9998288
, x: 0.63250405
y: 0.4943075
z: -0.88092214
visibility: 0.9998738
, x: 0.57338554
y: 0.5000746
z: -0.88228494
visibility: 0.9999145
, x: 0.8098198
y: 0.7292662
z: -0.28283337
visibility: 0.9974684
, x: 0.41927013
y: 0.7362031
z: -0.38274628
visibility: 0.99763423
, x: 0.9527502
y: 0.960179
z: -0.22109215
visibility: 0.4658201
, x: 0.3022582
y: 0.98350483
z: -0.46166486
visibility: 0.5355372
, x: 1.1147717
y:

# Checking the landmarks 

In [7]:
landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW]

x: 0.3022582
y: 0.98350483
z: -0.46166486
visibility: 0.5355372

In [8]:
landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER]

x: 0.41927013
y: 0.7362031
z: -0.38274628
visibility: 0.99763423

In [9]:
landmarks[mp_pose.PoseLandmark.RIGHT_WRIST]

x: 0.25096536
y: 1.3625742
z: -1.0252559
visibility: 0.18461625

# Calculating the angle 

In [18]:
def calculate_angle(a,b,c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle > 180.0:
        angle = 360-angle
        
    return angle

In [16]:
def hands():
    wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST].y]
    Elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW].y]
    shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].y]
    

# Driver code 

In [20]:
cap = cv2.VideoCapture(0)

# Curl counter variables
counter = 0 
stage = None

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            
            # Calculate angle
            angle = calculate_angle(shoulder, elbow, wrist)
            
            # Visualize angle
            cv2.putText(image, str(angle), 
                           tuple(np.multiply(elbow, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            
            # Curl counter logic
            if angle > 140:
                stage = "down"
            if angle < 90 and stage =='down':
                stage="up"
                counter +=1
                print(counter)
                       
        except:
            pass
        
        cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)
        
        # Rep data
        cv2.putText(image, 'REPS', (15,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), 
                    (10,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        # Stage data
        cv2.putText(image, 'STAGE', (65,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, stage, 
                    (60,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

1
2
3
4
